
---

# Extra Materials

## Keep at least $n$ books for every category:

In [ ]:
def rack_space_fixed(max_categories, max_budget, max_rack, books_df, fixed_space = 5, priority_constraint = 0.99, n_racks = 5, best_selling_cat = 5):
  #transform the budget in a number of books

  """ this feature decides how to fill the bookstore shelves and how much space to allocate to books and categories 
      based on demand and other factors and returns the list of books to put in each rack as a dictionnary.

    Args:
        max_categories (numeric): maximum number of categories to include.
        max_budget (numeric): maximum budget we have, that is the space available.
        max_rack (numeric): size of a single rack.
        books_df (dataframe): dataframe which contains important information about books
        priority_constraint (numeric): quantile which indicates the books priority (e.g.: a constraint of 0.99 means that we are considering only the 1% of most demanded books.)
        n_racks (numeric): number of racks to be given to the best selling category.
        best_selling_cat (numeric): number of categories best selled.
        

    Returns:
        (dict): books in each rack
    """
    
  ### SANITY CHECK
  assert max_categories <= len(books_df.Category.unique()), f'There are only {len(books_df.Category.unique())} categories, not {max_categories}'
  assert max_categories >= best_selling_cat, f'The number of categories {max_categories} should be strictly greater than the number of best selling categories {best_selling_cat} considered.'
  assert 0 < priority_constraint < 1, f'The priority constraint is the quantile considered as most important to keep in store, therefore should take values between 0 and 1.'

  ### INITIALIZING VARIABLES
  #a dictionary to store the books inventory for each rack
  rack_inventory = dict()

  #budget counter
  budget = 0

  #create df for the purpose of filling racks
  books_df = pd.merge(books, categories_data, on='Category').drop(['SalesRank', 'demand_abs', 'space_assigned'], axis = 1)

  #rename demand as single_book_demand
  books_df.rename(columns = {'demand':'single_book_demand'}, inplace = True)

  #sort categories by total category demand
  ###drop unneccessary columns
  books_df.drop(['gini_coef'], inplace = True, axis = 1)
  ###sort and keep the first n = max_categories items
  books_df.sort_values(by = 'demand_rel', inplace = True, ascending = False)

  cat_list = books_df.Category.unique()[:max_categories]

  # print('deb0', cat_list) #DEBUGGING
  
  ###SORTING BOOKS AND ASSIGNING SPACE
  
  budget_check = True

  while budget_check:

    # print('deb1', budget, 'First level reached.') #DEBUGGING

    for category in cat_list:
            #if categories end, loop end and all racks are filled without exausting the budget
                
      rack = list()

      #assign racks to category:

      if category in cat_list[:best_selling_cat]:
        rack_size = n_racks*max_rack
      elif categories_data.gini_coef.mean() > categories_data.loc[categories_data.Category == category].gini_coef.values[0]:
        rack_size = (n_racks // 2)*max_rack
      else:
        rack_size = max_rack   

      # print('deb2.1', category, budget) #DEBUGGING

      if budget >= max_budget:
        # print('deb2.2', budget_check, max_budget)
        budget_check = False
        break
      
      # print('deb2.3', budget_check) #DEBUGGING

      #order books by their demand
      #once categories are ordered, fill racks:
      book_list = books_df.loc[books_df.Category == category].sort_values(by = 'single_book_demand', ascending = False)        
              
      priority_quantile = books_df.single_book_demand.quantile(q = priority_constraint)

      # print('deb3', len(book_list)) #DEBUGGING
        
      for title in book_list.Title[:fixed_space]:

        single_title_demand = book_list.loc[book_list.Title == title].single_book_demand.values[0]
        value_check = single_title_demand > priority_quantile

        # print('deb4', single_title_demand, median_demand) #DEBUGGING

        if value_check and len(rack) < rack_size:
          # print('deb5', value_check, len(rack) < rack_size ) #DEBUGGING
          rack.append(title)
          #count budget: if budget is exausted, loop is ended and everything left out is left to be printed on demand
          budget +=1
          # print('deb666', budget, len(rack)) #DEBUGGING
          if budget >= max_budget:
            budget_check = False
            break
        else:
          break

      rack_inventory[category + ' max available space: ' +  str(rack_size)] = rack

      # print('deb7', budget, len(rack)) #DEBUGGING

      if budget >= max_budget:
        budget_check = False
        break
        # print('deb8', budget_check) #DEBUGGING

    if category == cat_list[len(cat_list) -1] and budget < max_budget:
      budget_check = False
      break
      
  spent_budget = sum([len(rack_inventory[label]) for label in rack_inventory.keys()])
  
  ### OUTPUT
  
  [print(label, f'  + Allocated space: {len(rack_inventory[label])}', sep = '\n') for label in rack_inventory.keys()]
  print('\n', '+ The net available space is: ', max_budget - spent_budget, sep = '')
  
  return rack_inventory

In [ ]:
rack_space_fixed(max_categories = 14, max_budget = 100, max_rack = 30, priority_constraint = 0.99, n_racks  = 5, books_df = books, best_selling_cat = 3)

In [ ]:
def rack_space_fixed2(max_categories, max_budget, max_rack, books_df, fixed_space = 5, priority_constraint = 0.99, n_racks = 5, best_selling_cat = 5):
  #transform the budget in a number of books

  """ this feature decides how to fill the bookstore shelves and how much space to allocate to books and categories 
      based on demand and other factors and returns the list of books to put in each rack as a dictionnary.

    Args:
        max_categories (numeric): maximum number of categories to include.
        max_budget (numeric): maximum budget we have, that is the space available.
        max_rack (numeric): size of a single rack.
        books_df (dataframe): dataframe which contains important information about books
        priority_constraint (numeric): quantile which indicates the books priority (e.g.: a constraint of 0.99 means that we are considering only the 1% of most demanded books.)
        n_racks (numeric): number of racks to be given to the best selling category.
        best_selling_cat (numeric): number of categories best selled.
        

    Returns:
        (dict): books in each rack
    """
    
  ### SANITY CHECK
  assert max_categories <= len(books_df.Category.unique()), f'There are only {len(books_df.Category.unique())} categories, not {max_categories}'
  assert max_categories >= best_selling_cat, f'The number of categories {max_categories} should be strictly greater than the number of best selling categories {best_selling_cat} considered.'
  assert 0 < priority_constraint < 1, f'The priority constraint is the quantile considered as most important to keep in store, therefore should take values between 0 and 1.'

  ### INITIALIZING VARIABLES
  #a dictionary to store the books inventory for each rack
  rack_inventory = dict()

  #budget counter
  budget = fixed_space*max_categories
  
  #create df for the purpose of filling racks
  books_df = pd.merge(books, categories_data, on='Category').drop(['SalesRank', 'demand_abs', 'space_assigned'], axis = 1)

  #rename demand as single_book_demand
  books_df.rename(columns = {'demand':'single_book_demand'}, inplace = True)

  #sort categories by total category demand
  ###drop unneccessary columns
  books_df.drop(['gini_coef'], inplace = True, axis = 1)
  ###sort and keep the first n = max_categories items
  books_df.sort_values(by = 'demand_rel', inplace = True, ascending = False)

  cat_list = books_df.Category.unique()[:max_categories]

  # print('deb0', cat_list) #DEBUGGING
  
  ###SORTING BOOKS AND ASSIGNING SPACE
  
  budget_check = True

  while budget_check:

    # print('deb1', budget, 'First level reached.') #DEBUGGING

    for category in cat_list:
            #if categories end, loop end and all racks are filled without exausting the budget
                
      rack = list()

      #assign racks to category:

      if category in cat_list[:best_selling_cat]:
        rack_size = n_racks*max_rack
      elif categories_data.gini_coef.mean() > categories_data.loc[categories_data.Category == category].gini_coef.values[0]:
        rack_size = (n_racks // 2)*max_rack
      else:
        rack_size = max_rack   

      # print('deb2.1', category, budget) #DEBUGGING

      if budget >= max_budget:
        # print('deb2.2', budget_check, max_budget)
        budget_check = False
        break
      
      # print('deb2.3', budget_check) #DEBUGGING

      #order books by their demand
      #once categories are ordered, fill racks:
      book_list = books_df.loc[books_df.Category == category].sort_values(by = 'single_book_demand', ascending = False)        
              
      priority_quantile = books_df.single_book_demand.quantile(q = priority_constraint)

      # print('deb3', len(book_list)) #DEBUGGING
        
      for title in book_list.Title[(fixed_space -1):]:

        single_title_demand = book_list.loc[book_list.Title == title].single_book_demand.values[0]
        value_check = single_title_demand > priority_quantile

        # print('deb4', single_title_demand, median_demand) #DEBUGGING

        if value_check and len(rack) < rack_size:
          # print('deb5', value_check, len(rack) < rack_size ) #DEBUGGING
          rack.append(title)
          #count budget: if budget is exausted, loop is ended and everything left out is left to be printed on demand
          budget +=1
          # print('deb666', budget, len(rack)) #DEBUGGING
          if budget >= max_budget:
            budget_check = False
            break
        else:
          break

      rack_inventory[category + ' max available space: ' +  str(rack_size)] = rack

      # print('deb7', budget, len(rack)) #DEBUGGING

      if budget >= max_budget:
        budget_check = False
        break
        # print('deb8', budget_check) #DEBUGGING

    if category == cat_list[len(cat_list) -1] and budget < max_budget:
      budget_check = False
      break
      
  spent_budget = sum([len(rack_inventory[label]) for label in rack_inventory.keys()])
  
  ### OUTPUT
  
  [print(label, f'  + Allocated space: {len(rack_inventory[label])}', sep = '\n') for label in rack_inventory.keys()]
  print('\n', '+ The net available space is: ', max_budget - spent_budget, sep = '')
  
  return rack_inventory

In [ ]:
rack_space_fixed2(max_categories = 14, max_budget = 930, max_rack = 30, priority_constraint = 0.99, n_racks  = 5, books_df = books, best_selling_cat = 3)